In [1]:
# Import libraries
import cv2
import csv
from datetime import datetime
import mediapipe as mp
import numpy as np
from scipy.stats import zscore

/Users/hayaomiyazaki/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [7]:
# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# Initialize video capture
cap = cv2.VideoCapture(0)

# Set the FPS
cap.set(cv2.CAP_PROP_FPS, 60)

# Indices for the nose tip and chin landmarks
NOSE_TIP_INDEX = 1
CHIN_INDEX = 152

# Set drawing specifications
landmark_specs = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)

# Create a CSV file with a timestamp in the filename
now = datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M%S_%f")
initial_timestamp_in_microseconds = (
    now.hour * 3_600_000_000 +   # Hours to microseconds
    now.minute * 60_000_000 +    # Minutes to microseconds
    now.second * 1_000_000 +     # Seconds to microseconds
    now.microsecond              # Already in microseconds
)
csv_filename = f"data/csv/face_landmarks_{timestamp}.csv"

# Peak detection variables
distances = []
peak_count = 0
interval_count = 0
Z_THRESHOLD = 2.22  # Z-score threshold for peak detection

with open(csv_filename, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['Frame_Count', 'Timestamp', 'Nose_Tip_X', 'Nose_Tip_Y', 'Chin_X', 'Chin_Y', 'Distance'])

    # Create Face Mesh object with specific settings
    with mp_face_mesh.FaceMesh(
        max_num_faces=1,
        refine_landmarks=True,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5
    ) as face_mesh:

        # Check if the video source is available
        if not cap.isOpened():
            print("Error: Could not open video source.")
        else:
            frame_count = 0
            # Process each video frame
            while cap.isOpened():
                success, image = cap.read()

                if not success:
                    print("Ignoring empty camera frame.")
                    continue

                # Convert the image to RGB for face mesh processing
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = face_mesh.process(image_rgb)

                # If face landmarks are detected
                if results.multi_face_landmarks:
                    for face_landmarks in results.multi_face_landmarks:
                        # Extract the landmark coordinates
                        nose_tip = face_landmarks.landmark[NOSE_TIP_INDEX]
                        chin = face_landmarks.landmark[CHIN_INDEX]

                        # Convert normalized coordinates to pixel values
                        h, w, _ = image.shape
                        nose_tip_coords = (int(nose_tip.x * w), int(nose_tip.y * h))
                        chin_coords = (int(chin.x * w), int(chin.y * h))

                        # Calculate the Euclidean distance
                        distance = round(
                            (abs(nose_tip_coords[0] - chin_coords[0])**2 + 
                             abs(nose_tip_coords[1] - chin_coords[1])**2)**0.5, 10
                        )

                        # Add distance to the list
                        distances.append(distance)
                        
                        # Get timestamp in microseconds
                        now = datetime.now()
                        timestamp_in_microseconds = (
                            now.hour * 3_600_000_000 +   # Hours to microseconds
                            now.minute * 60_000_000 +    # Minutes to microseconds
                            now.second * 1_000_000 +     # Seconds to microseconds
                            now.microsecond              # Already in microseconds
                        )
                        if initial_timestamp_in_microseconds > timestamp_in_microseconds:
                            timestamp_in_microseconds = timestamp_in_microseconds + 3_600_000_000 * 24 - initial_timestamp_in_microseconds
                        else:
                            timestamp_in_microseconds -= initial_timestamp_in_microseconds
                        
                        # Write coordinates to CSV
                        csv_writer.writerow([
                            frame_count, timestamp_in_microseconds,
                            nose_tip_coords[0], nose_tip_coords[1],
                            chin_coords[0], chin_coords[1],
                            distance
                        ])

                        # Draw the landmark points on the image
                        for coords in [nose_tip_coords, chin_coords]:
                            cv2.circle(image, coords, 5, (0, 255, 0), -1)

                        # Draw lines between the points
                        cv2.line(image, nose_tip_coords, chin_coords, (255, 0, 0), 2)

                        # Real-time Z-Score calculation for peak detection
                        if len(distances) > 12:  # Start detecting after 12 frames
                            z_scores = zscore(distances[-12:])  # Calculate Z-score for the last 30 distances

                            if interval_count > 0:
                                interval_count -= 1
                            
                            elif abs(z_scores[-1]) > Z_THRESHOLD:
                                peak_count += 1
                                interval_count = 10
                                print(f"Peak detected! Total peaks: {peak_count}")

                # Display the output
                cv2.imshow("Face Landmarks Detection", cv2.flip(image, 1))

                frame_count += 1

                # Break the loop if 'q' is pressed
                if cv2.waitKey(16) == ord('q'):
                    break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

print(f"Landmarks data saved to {csv_filename}")

Peak detected! Total peaks: 1
Peak detected! Total peaks: 2
Peak detected! Total peaks: 3
Peak detected! Total peaks: 4
Landmarks data saved to data/csv/face_landmarks_20241112_125953_212479.csv
